In [1]:
import os
import sys

from google.cloud import bigquery
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.config.settings import BIGQUERY_DATASET, BIGQUERY_RAW_TABLE, GCP_PROJECT_ID

# Load data

## Load with old method

In [2]:
CREDENTIALS_PATH = '../credentials/sa-athlete-dashboard.json'
TABLE_ID = f'{GCP_PROJECT_ID}.{BIGQUERY_DATASET}.{BIGQUERY_RAW_TABLE}'
print(TABLE_ID)

client = bigquery.Client.from_service_account_json(
    CREDENTIALS_PATH, project='athlete-dashboard-467718'
)

athlete-dashboard-467718.strava_data.strava_activities_raw


In [ ]:
table = client.get_table(TABLE_ID)
print(f'Tabelle gefunden: {TABLE_ID}')
print(f'Spalten: {[schema.name for schema in table.schema]}')
print(f'Zeilen: {table.num_rows}')

# --- 5. Kleine Abfrage, um Daten zu testen ---
query = f'SELECT * FROM `{TABLE_ID}`'
print('🔹 Starte Abfrage:', query)

job = client.query(query)
df = job.result().to_dataframe()  # Ergebnisse in Pandas DataFrame

print('✅ Abfrage abgeschlossen')

Tabelle gefunden: athlete-dashboard-467718.strava_data.strava_activities_raw
Spalten: ['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time', 'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id', 'start_date', 'start_date_local', 'timezone', 'utc_offset', 'location_city', 'location_state', 'location_country', 'achievement_count', 'kudos_count', 'comment_count', 'athlete_count', 'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed', 'max_speed', 'average_cadence', 'average_temp', 'average_watts', 'max_watts', 'weighted_average_watts', 'device_watts', 'kilojoules', 'has_heartrate', 'average_heartrate', 'max_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id', 'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed', 'athlete_id', 'athlete_resource_state', 'map_id', 'map_summary_polyli

c:\Users\xaver\Projekte\Athlete-Dashboard\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


✅ Abfrage abgeschlossen, erste Zeilen:


## Load with loader

In [11]:
from src.config.settings import load_strava_configs


CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN = load_strava_configs('local')

ImportError: cannot import name 'load_strava_configs' from 'src.config.settings' (c:\Users\xaver\Projekte\Athlete-Dashboard\src\config\settings.py)

In [7]:
# TODO: Implement loading with loader
from src.elt.extractors.strava import StravaExtractor, get_fresh_access_token

access_token = get_fresh_access_token(CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN)


extractor = StravaExtractor(access_token)
all_activities = extractor.fetch_all_activities()

Refreshing Strava access token...
Successfully refreshed access token.
Start fetching all activities.
All activities fetched. Found 283 activities.


In [8]:
df_all_activities_raw = pd.json_normalize(all_activities)
df_all_activities_raw.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state
0,2,Easy Run,10095.0,3507,3507,83.0,Run,Run,NaN,16171198076,...,garmin_ping_491922155245,False,0,0,False,133094316,1,a16171198076,qeluHwo|j@DbC?|@DlCEn@If@BZCf@?j@SbDKx@Qz@c@xA...,2
1,2,Cool Down,3184.9,1049,1092,57.0,Run,Run,NaN,16157747972,...,garmin_ping_491481976946,False,0,0,False,133094316,1,a16157747972,wcjuHmdbk@s@^e@L}DbBiA\IHQXA@?EWFYTe@Lw@XoAj@o...,2
2,2,Intervals,9231.1,2519,2524,1.0,Run,Run,NaN,16157747867,...,garmin_ping_491481965492,False,1,0,False,133094316,1,a16157747867,s~iuHq|ak@ZMLQFOHm@?g@Ia@QYOKWE{@L}AJOFOJQVGTC...,2
3,2,Warm Up,3193.5,950,988,22.0,Run,Run,NaN,16157747870,...,garmin_ping_491481939650,False,0,0,False,133094316,1,a16157747870,kcluHiq}j@Xk@L_@LSlAwCH}AW}BIoBW_DM{@KmAUiB_@_...,2
4,2,Threshold Run,11675.6,3211,3214,91.0,Run,Run,NaN,16135627093,...,garmin_ping_490767249442,False,0,0,False,133094316,1,a16135627093,seluHan|j@L|C?lADd@KjAExAMx@GbBIbAOjA_@|@c@vAS...,2


In [9]:
import re

# Data Cleaning for bigquery
df_all_activities = df_all_activities_raw.copy()

# Replace all invalid characters with underscores
df_all_activities.columns = [
    re.sub(r'[^a-zA-Z0-9_]', '_', col) for col in df_all_activities.columns
]

# Clean data

In [ ]:
df.head()

# Data loading

In [ ]:
# TODO: Implement writing into  new bigquery table
# Use table strava_activities_cleaned 